In [ ]:
# latest update
# June 27, 2017
# Changes done by: Jimmy Vivas

# This code was developed by Jimmy Vivas for Aspen Technology
# For Shell case, they created one file per month and a folder per tag

# The code will take all files within sub folders but it wont process files located in the root folder
# The colde will take the file name as a tagname and look for that tagname in a tag dictionary file
# Then it will add the tagname and tag description to the CSV file for each file wihtin sub folders
# The code will merge all individual csv files (monthly files) to be one single file

In [ ]:
from os import listdir
from os import walk
import pandas as pd
import math
import numpy as np

In [ ]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

CSVPath1 = '/Users/jvivas/Documents/Aspen/Shell/to be processed'
CSVMetaDataFile = '/Users/jvivas/Documents/Aspen/Shell/Metadata/Data Dictionary.csv'
CSVPathProcessed = '/Users/jvivas/Documents/Aspen/Shell/Processed'

# Windows Path
# CSVPath1 = 'C:/Users/jvivas/Dropbox/Mtell Customer Projects/XOM BayTown RHC MEA Tower Foaming/Sensor Data/' \
#           'To be Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [ ]:
def GetFiles(CSVPath):
    CSVFileList = []
    NanPHDTagList = []
    CSVFileListAll = listdir(CSVPath)

    m = len(CSVFileListAll)

    for i in range(m):
        fileNameStr = CSVFileListAll[i]
        fileStr = fileNameStr.split('.')[0]
        fileExt = fileNameStr.split('.')[1]
        if fileExt == "csv":
            CSVFileList.append(fileNameStr)

    n = len(CSVFileList)
    return(n, CSVFileList)

# Uncomment following cell for debugging purposes

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# np.set_printoptions(threshold=np.nan)

In [ ]:
def LoadCSV(FileAndPath, file):
    print ('Loading ' + file)
    df2 = pd.read_csv(FileAndPath+'/'+file, low_memory=False)
    return(df2)

In [ ]:
def GetTagName(CSVFileName):
    IndexStr = CSVFileName.replace('_', 'OK', 2).find('_') - 2
    TagName = CSVFileName[: IndexStr]
    return(TagName)

# This is the Main Code to execute

In [ ]:
print ('Loading Tagname Metadata')
TagMetaDataDF = pd.read_csv(CSVMetaDataFile, low_memory=False)

FolderList = next(walk(CSVPath1))[1]
FolderListNumber = len(FolderList)

for j in range(FolderListNumber):
    CSVFinalPath = CSVPath1 + '/' + FolderList[j]
    
    print ('Getting files within folder: ' + FolderList[j])
    n, CSVFileList = GetFiles(CSVFinalPath)

    i = 0
    for i in range(n - 1):
        CSVFile = CSVFileList[i]
        # CSVFileWithPath = CSVPath1 + "/" + CSVFileList[i]

        df2 = LoadCSV(CSVFinalPath, CSVFile)
        print ('Getting TagName')
        TagName =  GetTagName(CSVFile)
        print('Getting TagDescription')
        TagDescription = TagMetaDataDF.loc[TagMetaDataDF['Tag Name'] == TagName, 'Description'].values[0]
        print ('Adding Tag name and description to CSV file')

        df2['TagName'] = TagName
        df2['Description'] = TagDescription

        df2.to_csv(CSVPathProcessed + '/' + str(CSVFile.replace('.csv', '')) + '_Formatted.csv', index=False)
        if i == 0:
            df2_concatenated = df2.copy()
        else:
            df2_concatenated = df2_concatenated.append(df2)
    df2_concatenated.to_csv(CSVPathProcessed + '/' + str(CSVFile.replace('.csv', '')) + '_Concatenated.csv', index=False)
    print ('Tagname: ' + TagName + ' Processed')
